In [1]:
import winsound
from scipy.spatial import distance as dist
import cv2
import dlib
import imutils 
from imutils import face_utils

In [6]:
frequency = 1500    # frequency of sound
duration = 1000    # duration of 1 beep sound
earThresh = 0.2    
earFrames = 50    # min consecutive frames for eye closures to consider for drowsiness
count = 0

In [3]:
# face detector similar to haarcascade
detector = dlib.get_frontal_face_detector() 

# outputs a set of point locations that define the pose of the object - facial landmark predictor
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

In [4]:
# EAR refers to the aspect ratio of the eye region i.e ratio of the horizontal length to the vertical length

(lStart , lEnd) = face_utils.FACIAL_LANDMARKS_IDXS['left_eye']
(rStart , rEnd) = face_utils.FACIAL_LANDMARKS_IDXS['right_eye']

print('Left Eye start and end coordinates: ', lStart , lEnd)
print('Right Eye start and end coordinates: ', rStart , rEnd)

Left Eye start and end coordinates:  42 48
Right Eye start and end coordinates:  36 42


In [5]:
def eyeAspectRatio(eye):
    A = dist.euclidean(eye[1] , eye[5])
    B = dist.euclidean(eye[2] , eye[4])
    C = dist.euclidean(eye[0] , eye[3])
    ear = (A + B)/(2*C)
    return ear

In [9]:
cap = cv2.VideoCapture(0)

while True:
    # load the input image, resize it, and convert it to grayscale
    res , frame = cap.read()
    if res == False:
        print("Can't receive frame (stream end?). Exiting ...")
        break
        
    frame = imutils.resize(frame , width=750)
    gray = cv2.cvtColor(frame , cv2.COLOR_BGR2GRAY)
     
    # detect faces in the grayscale image - coordinates of face (x , y , w , h) 
    rects = detector(gray , 0)  # second argument indicates that we should upsample the image 
    
    for rect in rects:
        
        # determine the facial landmarks for the face region
        shape = predictor(gray , rect)
        # convert the landmark (x, y)-coordinates to a NumPy array
        shape = face_utils.shape_to_np(shape)
        
        leftEye = shape[lStart:lEnd]
        rightEye = shape[rStart:rEnd]
        
        # compute eye aspect ratio for left and right eye
        leftEAR = eyeAspectRatio(leftEye)
        rightEAR = eyeAspectRatio(rightEye)
        
        # if the avarage eye aspect ratio of left and right eye less than 0.2, the status is sleeping
        ear = (leftEAR + rightEAR)/2
        
        # Finds the convex hull of the point sets
        leftEyeHull = cv2.convexHull(leftEye)
        rightEyeHull = cv2.convexHull(rightEye)
        
        # Draws contours outlines
        cv2.drawContours(frame , [leftEyeHull] , -1 , (0,0,255) , 1, )
        cv2.drawContours(frame , [rightEyeHull] , -1 , (0,0,255) , 1)
        
        if ear < earThresh:
            count += 1
            
            if count >= earFrames:
                cv2.putText(frame, 'DROWSINESS DETECTED', (100,150), cv2.FONT_ITALIC, 1, (0,0,255), 1)
                winsound.Beep(frequency , duration)
                
    cv2.imshow('Frame' , frame)
    if cv2.waitKey(1) == 27:         # break if esc key is pressed
        break

# releasing the VideoCapture object
cap.release()
cv2.destroyAllWindows()

In [10]:
cap.release()
cv2.destroyAllWindows()